In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from mcdrift import *

In this experiment, we would like to compare the performance of P-CDM and NP-CDM. We set the following parameters:

1. $N = 5$ (number of states)

2. $T = 10,000$ (sequence length)

3. $t^* = 2,000$ (time abrupt change occurs)

4. $L = 1,000$ (number of observations guaranteed to come from $P_0$)

5. $W \in \{2, 5, 10, 50, 100\}$ (subsequence length)

6. $K \in \{1, 2, 5, 10, 20, 50\}$ (detection threshold)

We randomly generate $P_0$ and $P_1$ from the standard uniform distribution and normalize such that each row sums to 1. We set $\pi = (0.2, 0.2, 0.2, 0.2, 0.2)$.

Also, we estimate the time of abrupt change using the center of the predicted interval. For example, when $W = 10$ and the change is detected in $s_k$ with $91 \leq k \leq 100$, the estimated time is simply $(100+91)/2 = 95.5$.

We repeat the experiments $500$ times and take the average of the estimated times.

In [86]:
N = 5
T = 10000
tstar = 2000
L = 1000
Ws = [2, 5, 10, 50, 100]
Ks = [1, 2, 5, 10, 20, 50]
pi = np.array([1/5, 1/5, 1/5, 1/5, 1/5])

result_dict_p = {}
result_dict_np = {}


for W in Ws:
    for K in Ks:
        result_dict_p[(W,K)] = 0
        result_dict_np[(W,K)] = 0

seed = 2023
n_rep = 500

In [87]:
np.random.seed(seed)

for _ in tqdm(range(n_rep)):
    p0 = np.random.rand(N,N)
    p0 = p0/p0.sum(axis=1,keepdims=1)
    p1 = np.random.rand(N,N)
    p1 = p1/p1.sum(axis=1,keepdims=1)

    seq_sim = simulate_mc(pi, p0, tstar)
    init_vec = np.zeros(N)
    init_vec[seq_sim[-1]] = 1
    seq_sim_2 = simulate_mc(init_vec, p1, T - tstar + 1)
    seq_comb = seq_sim + seq_sim_2[1:]
    
    for W in Ws:
        for K in Ks:
            pcdm_res = pcdm(seq_comb, W, p0, p1, K)
            npcdm_res = npcdm(seq_comb, W, N, L, K)
            if type(pcdm_res) == str:
                pcdm_res = T/W
            if type(npcdm_res) == str:
                npcdm_res = T/W
            result_dict_p[(W,K)] += (2 * W * pcdm_res + W + 1)/(2 * n_rep)
            result_dict_np[(W,K)] += (2 * W * npcdm_res + W + 1)/(2 * n_rep)

100%|██████████| 500/500 [15:50<00:00,  1.90s/it]


In [88]:
result_dict_p

{(2, 1): 6.456000000000002,
 (2, 2): 28.020000000000003,
 (2, 5): 590.9360000000001,
 (2, 10): 1725.2879999999986,
 (2, 20): 2105.9560000000006,
 (2, 50): 2433.6559999999995,
 (5, 1): 28.38999999999997,
 (5, 2): 221.85000000000014,
 (5, 5): 1858.690000000007,
 (5, 10): 2073.719999999995,
 (5, 20): 2155.690000000004,
 (5, 50): 2400.890000000003,
 (10, 1): 132.62000000000012,
 (10, 2): 1106.3999999999978,
 (10, 5): 2043.039999999992,
 (10, 10): 2116.1400000000026,
 (10, 20): 2240.7800000000043,
 (10, 50): 2614.419999999999,
 (50, 1): 1945.89999999999,
 (50, 2): 2074.600000000011,
 (50, 5): 2226.000000000009,
 (50, 10): 2477.400000000009,
 (50, 20): 2980.6000000000076,
 (50, 50): 4489.400000000006,
 (100, 1): 2044.500000000028,
 (100, 2): 2150.4999999999864,
 (100, 5): 2450.499999999997,
 (100, 10): 2950.499999999984,
 (100, 20): 3950.4999999999677,
 (100, 50): 6951.699999999949}

In [89]:
result_dict_np

{(2, 1): 1005.4239999999922,
 (2, 2): 1030.6959999999988,
 (2, 5): 1132.988,
 (2, 10): 1493.4159999999997,
 (2, 20): 2202.3119999999976,
 (2, 50): 3248.0160000000037,
 (5, 1): 1048.8499999999945,
 (5, 2): 1140.930000000001,
 (5, 5): 1527.2099999999994,
 (5, 10): 2228.030000000003,
 (5, 20): 3414.1399999999967,
 (5, 50): 4088.5299999999957,
 (10, 1): 1096.759999999996,
 (10, 2): 1261.5600000000015,
 (10, 5): 1874.0600000000031,
 (10, 10): 2994.9599999999996,
 (10, 20): 3910.6000000000004,
 (10, 50): 4429.560000000001,
 (50, 1): 1459.0999999999995,
 (50, 2): 1969.9,
 (50, 5): 3432.8000000000015,
 (50, 10): 4368.900000000003,
 (50, 20): 4779.500000000008,
 (50, 50): 5929.600000000016,
 (100, 1): 1842.0999999999967,
 (100, 2): 2751.1000000000117,
 (100, 5): 4170.300000000015,
 (100, 10): 4747.500000000004,
 (100, 20): 5515.099999999999,
 (100, 50): 7843.299999999994}